In [1]:
from openff.evaluator.utils import setup_timestamp_logging

setup_timestamp_logging()

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:
import pathlib

from openff.evaluator.datasets import PhysicalPropertyDataSet

data_set_path = "filtered_data_set_alkanes.json"

data_set = PhysicalPropertyDataSet.from_json(data_set_path)

In [3]:
data_set.to_pandas().head()

,Id,Temperature (K),Pressure (kPa),Phase,N Components,Component 1,Role 1,Mole Fraction 1,Exact Amount 1,Density Value (g / ml),Density Uncertainty (g / ml),EnthalpyOfVaporization Value (kJ / mol),EnthalpyOfVaporization Uncertainty (kJ / mol),Source
0,5ca3d13d93594783984ca9776d286949,298.15,101.325,Liquid + Gas,1,CCCCCC,Solvent,1.0,None,NaN,NaN,31.52,0.028,10.1063/1.555942
1,ab6c5784ea6d4b109377e0940f49bdcb,298.15,101.325,Liquid + Gas,1,CCCCCCCC,Solvent,1.0,None,NaN,NaN,41.60,0.000,10.1021/je960089h
2,5392ede1c7e640129eab899ee9a898f5,298.15,101.325,Liquid + Gas,1,CCCCCCCCCC,Solvent,1.0,None,NaN,NaN,51.40,0.000,10.1021/je00014a009
3,d947ea7a12524100a1ae7681ecfd206f,298.15,101.325,Liquid + Gas,1,CCCCCCCCCCCC,Solvent,1.0,None,NaN,NaN,61.30,0.300,10.1021/je960089h
4,b52b79995f17461e8e57b3f69b452720,298.15,101.325,Liquid + Gas,1,CCCCCCCCCCCCCCC,Solvent,1.0,None,NaN,NaN,76.40,0.300,10.1021/je960089h


In [5]:
from openff.evaluator.forcefield import SmirnoffForceFieldSource

force_field_path = "sage2.2.1-alkanes-filtered.offxml"
force_field_source = SmirnoffForceFieldSource.from_path(force_field_path)

In [7]:
from openff.evaluator.client import RequestOptions

# Create an options object which defines how the data set should be estimated.
estimation_options = RequestOptions()
# Specify that we only wish to use molecular simulation to estimate the data set.
estimation_options.calculation_layers = ["SimulationLayer"]

# Add our custom schemas, specifying that the should be used by the 'SimulationLayer'
# estimation_options.add_schema("SimulationLayer", "Density", density_schema)
# estimation_options.add_schema("SimulationLayer", "EnthalpyOfVaporization", h_vap_schema)

In [ ]:
import os

from openff.evaluator.backends import ComputeResources
from openff.evaluator.backends.dask import DaskLocalCluster

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

calculation_backend = DaskLocalCluster(
    number_of_workers=1,
    resources_per_worker=ComputeResources(
        number_of_threads=1,
        number_of_gpus=1,
        preferred_gpu_toolkit=ComputeResources.GPUToolkit.CUDA,
    ),
)
calculation_backend.start()

In [ ]:
from openff.evaluator.server import EvaluatorServer

evaluator_server = EvaluatorServer(calculation_backend=calculation_backend)
evaluator_server.start(asynchronous=True)

In [ ]:
from openff.evaluator.client import EvaluatorClient

evaluator_client = EvaluatorClient()

request, exception = evaluator_client.request_estimate(
    property_set=data_set,
    force_field_source=force_field_source,
    options=estimation_options,
)

assert exception is None

                                  Id  Temperature (K)  Pressure (kPa)  \
0   9a7cb388dbde400bb1219a005584e562           298.15         101.325   
1   1cee9e6b757c44b4baddecc9d0c925b6           298.15         101.325   
2   188084f68b3a480b8d34f766dd5a5315           298.15         101.325   
3   759a164c1bd84f63aa8683e9eef948d9           298.15         101.325   
4   41c56af6ea8b494f98aa0d53c91f293f           298.15         101.325   
5   5ca3d13d93594783984ca9776d286949           298.15         101.325   
6   ab6c5784ea6d4b109377e0940f49bdcb           298.15         101.325   
7   5392ede1c7e640129eab899ee9a898f5           298.15         101.325   
8   d947ea7a12524100a1ae7681ecfd206f           298.15         101.325   
9   b52b79995f17461e8e57b3f69b452720           298.15         101.325   
10  54166af9ba5c4bbab4d2920e6e2ae77a           298.15         101.325   
11  1ee4936ef9a64063b4619cd1f8b6a5d4           298.15         101.325   
12  b325b367fdf547a39d81196161d860eb           298.

In [ ]:
# Wait for the results.
results, exception = request.results(synchronous=True, polling_interval=30)
assert exception is None

In [ ]:
print(len(results.queued_properties))

print(len(results.estimated_properties))

print(len(results.unsuccessful_properties))
print(len(results.exceptions))

                                  Id  Temperature (K)  Pressure (kPa)  \
0   9a7cb388dbde400bb1219a005584e562           298.15         101.325   
1   1cee9e6b757c44b4baddecc9d0c925b6           298.15         101.325   
2   188084f68b3a480b8d34f766dd5a5315           298.15         101.325   
3   759a164c1bd84f63aa8683e9eef948d9           298.15         101.325   
4   41c56af6ea8b494f98aa0d53c91f293f           298.15         101.325   
5   5ca3d13d93594783984ca9776d286949           298.15         101.325   
6   ab6c5784ea6d4b109377e0940f49bdcb           298.15         101.325   
7   5392ede1c7e640129eab899ee9a898f5           298.15         101.325   
8   d947ea7a12524100a1ae7681ecfd206f           298.15         101.325   
9   b52b79995f17461e8e57b3f69b452720           298.15         101.325   
10  54166af9ba5c4bbab4d2920e6e2ae77a           298.15         101.325   
11  1ee4936ef9a64063b4619cd1f8b6a5d4           298.15         101.325   
12  b325b367fdf547a39d81196161d860eb           298.

In [ ]:
results.estimated_properties.json("estimated_data_set.json", format=True)

                                  Id  Temperature (K)  Pressure (kPa)  \
0   9a7cb388dbde400bb1219a005584e562           298.15         101.325   
1   1cee9e6b757c44b4baddecc9d0c925b6           298.15         101.325   
2   188084f68b3a480b8d34f766dd5a5315           298.15         101.325   
3   759a164c1bd84f63aa8683e9eef948d9           298.15         101.325   
4   41c56af6ea8b494f98aa0d53c91f293f           298.15         101.325   
5   5ca3d13d93594783984ca9776d286949           298.15         101.325   
6   ab6c5784ea6d4b109377e0940f49bdcb           298.15         101.325   
7   5392ede1c7e640129eab899ee9a898f5           298.15         101.325   
8   d947ea7a12524100a1ae7681ecfd206f           298.15         101.325   
9   b52b79995f17461e8e57b3f69b452720           298.15         101.325   
10  54166af9ba5c4bbab4d2920e6e2ae77a           298.15         101.325   
11  1ee4936ef9a64063b4619cd1f8b6a5d4           298.15         101.325   
12  b325b367fdf547a39d81196161d860eb           298.